In [16]:
# Recuperation des arcs sous la forme x_(i,j) où i est l'origine et j la destination
function getEdge(kep)
    edge_list = Array{Array{Int64,1},1}()
    w = zeros(Int8, nv(kep), nv(kep))
    for (ind,arc) in enumerate(edges(kep))
            push!(edge_list,[src(arc), dst(arc)])
        w[src(arc), dst(arc)] = weights[ind]
    end
    return edge_list, w
end

getEdge(pool)

#Recuperation de tous les chemins de longueur L
function getPath_L(pool)
    path_L = filter(x->length(x)==L,simplecycles_hawick_james(pool))
    return path_L
end

getPath_L (generic function with 1 method)

In [17]:
function edgeformulation(pool)    
    # Le probleme

    model=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(model, "OutputFlag", 0) #???? parametre a modifier

    # Les donnees
    path_L = getPath_L(pool)
    #Ensemble des arcs sous la forme [i,j]
    edge_list, w = getEdge(pool); #Penser a ajouter la fonction PROBLEME_SANS_REFORMULATION

    #Nombre de sommets
    E = ne(pool)
    V = nv(pool)

    #Les variables
    @variable(model, x[i in 1:V, j in 1:V], Bin)

    #La fonction objective
    @objective(model, Max, sum(sum(w[i,j] * x[i,j] for i in 1:V) for j in 1:V))

    #Les contraintes
    for i in 1:V
        for j in 1:V
            if !([i,j] in edge_list)
                @constraint(model, x[i,j]==0)
            end
        end
    end

    #Premiere contrainte
    for i in 1:V
        E_1 = [] #Ensemble des arcs ou i est le depart
        E_2 = [] #Ensemble des arcs ou i est l arrivee
        #Recuperer les arcs ou il y a i en extremite
        for e in edge_list
            if e[1] == i
                push!(E_1, e[2])
            end
            if e[2] == i
                push!(E_2, e[1])
            end
        end
        #La contrainte sur le sommet i
        @constraint(model, sum(x[e, i] for e in E_2) == sum(x[i, e] for e in E_1) )
    end

    #Deuxieme contrainte
    for i in 1:V
        E_i = [] #Ensemble des arcs ou i est l arrivee
        #Recuperer les arcs ou il y a i en extremite
        for e in edge_list
            if e[1] == i
                push!(E_i, e[2])
            end
        end
        #La contrainte sur le sommet i
        @constraint(model, sum(x[i, e] for e in E_i) <= 1)
    end

    #Troisieme contrainte
    for p in path_L #Pour tous les chemins de longueur L
        @constraint(model, sum(x[p[j], p[j+1]] for j in 1:L-1) <= L-1)
    end
    
    #Résolution du problème
    optimize!(model)
    
    #Recuperation solution
    x_value = JuMP.value.(x)
    solu_x = []
    for i = 1:V
        for j = 1:V
            if(x_value[i,j]==1)
                push!(solu_x, [i,j])
            end
        end
    end
    
    return(objective_value(model),solu_x)
end

In [ ]:
function formulationIP(pool)
    # Initialisation du modèle
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    # Recupération des arcs des cycles de longueur < L
    cycles = get_cycles_arcs(get_cycles(pool,L))
    nb_cycles = size(cycles,1)
    # Definition et calcul des "colonnes" associées à chaque cycle
    # 1er element -> cout du cycle
    # 2 - nb(arcs) elements -> vaut 1 si l arc est dans le cycle, 0 sinon
    col_pool = Array{Array{Int,1},1}(undef,nb_cycles)
    cost = []
    for ind in 1:nb_cycles
        cost = sum(weights[a] for a in cycles[ind])
        val_arcs = zeros(Int,ne(pool))
        val_arcs[cycles[ind]].= 1
        col_pool[ind] = vcat(cost,val_arcs)
    end
    
    # Definition des variables
    # variable cycle 
    # z_c = 1 si cycle c choisi, 0 sinon
    @variable(model,z[c in 1:nb_cycles],Bin)
    
    # Contraintes
    # Si une paire est dans un cyle, alors le cycle contient un arc dont la paire est à l'origine
    # definition de  y_c_i = 1 si sommet i dans le cycle c, 0 sinon
    y = zeros(Int, size(col_pool,1), nv(pool))
    for c in 1:nb_cycles
        for v in 1:nv(pool)
            if(!isempty(origine[v]))
                y[c,v] = sum(col_pool[c][k+1] for k in origine[v])
            end
        end
    end
    
    paire = Array{ConstraintRef,1}(undef,nv(pool))
    for v in 1:nv(pool)
        paire[v] = @constraint(model,sum(y[c,v]*z[c] for c in 1:nb_cycles)<=1)
        set_name(paire[v],"paire_$v")
    end
    
    # Fonction objective
    @objective(model,Max,sum(col_pool[c][1]*z[c] for c in 1:nb_cycles))
    
    optimize!(model)

    # Si le probleme est resolu à l'optimalité, on renvoie la solution 
    # ainsi que les arcs
    if JuMP.termination_status(model)==MOI.OPTIMAL
        x = Array{Float32,1}(undef,ne(pool))
        for edge in 1:ne(pool)
            x[edge] = sum(col_pool[c][edge+1]*JuMP.value.(z)[c] for c in 1:nb_cycles)
        end
        return JuMP.objective_value(model),x
    else
        return "PROBLEME NON RESOLU"
    end
end